Goal: apply a DL RNN model on all patients one by one
- Preprocess with CustomTransformer

# Imports, variables, functions

In [2]:
#import scipy.io
#from pyedflib import highlevel
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
#from itertools import groupby
import csv
#import pickle
#from scipy.signal import butter, sosfilt, sosfiltfilt, sosfreqz
#from scipy.signal import freqz, iirnotch, filtfilt
#from sklearn.preprocessing import MinMaxScaler
#from sklearn.base import TransformerMixin, BaseEstimator
import random
from sklearn.model_selection import cross_validate
#from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
#from scipy.fftpack import rfft
#from sklearn.metrics import accuracy_score
import tensorflow as ts
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from plot_keras_history import plot_history

In [3]:
sample_rate = sampling_rate = 256
sec = 10
len_window = sample_rate*sec
overlap = 5
threshold=2

In [3]:
#Importing ten patients preprocessed with Custom Transformer
patient1=pd.read_csv('../raw_data/preprocessed/preprocessed_patient_1.csv',index_col=[0])

In [4]:
patient2=pd.read_csv('../raw_data/preprocessed/preprocessed_patient_2.csv',index_col=[0])

In [5]:
patient3=pd.read_csv('../raw_data/preprocessed/preprocessed_patient_3.csv',index_col=[0])

In [6]:
patient6=pd.read_csv('../raw_data/preprocessed/preprocessed_patient_6.csv',index_col=[0])

In [4]:
patient10=pd.read_csv('../raw_data/preprocessed/preprocessed_patient_10.csv',index_col=[0])

In [ ]:
patient13=pd.read_csv('../raw_data/preprocessed/preprocessed_patient_13.csv',index_col=[0])

In [ ]:
patient14=pd.read_csv('../raw_data/preprocessed/preprocessed_patient_14.csv',index_col=[0])

In [ ]:
patient18=pd.read_csv('../raw_data/preprocessed/preprocessed_patient_18.csv',index_col=[0])

In [11]:
patient19=pd.read_csv('../raw_data/preprocessed/preprocessed_patient_19.csv',index_col=[0])

In [ ]:
patient78=pd.read_csv('../raw_data/preprocessed/preprocessed_patient_78.csv',index_col=[0])

In [ ]:
#patients = [2, 3, 6, 10, 1, 13, 14, 18, 19, 78]

In [8]:
df = pd.concat([patient1,patient2,patient3,patient6,patient10])

In [9]:
df

,eeg fp1-ref,eeg fp2-ref,eeg f3-ref,eeg f4-ref,eeg c3-ref,eeg c4-ref,eeg p3-ref,eeg p4-ref,eeg o1-ref,eeg o2-ref,eeg f7-ref,eeg f8-ref,eeg t3-ref,eeg t4-ref,eeg t5-ref,eeg t6-ref,eeg fz-ref,eeg cz-ref,eeg pz-ref,is_seizure_target
0,-0.062060,0.059722,-0.441541,-0.136232,-0.373716,-0.246746,-0.220026,-0.255515,-0.316143,-0.448289,-0.235465,-0.163934,-0.267007,-0.311155,-0.179917,-0.524558,-0.239511,-0.424504,-0.593197,0
1,-0.725566,-0.742145,-0.337565,-0.144298,-0.273719,-0.086159,0.155921,-0.416544,0.119535,-0.823691,-0.575949,-0.513936,-0.507341,-0.620216,-0.310742,-1.131727,-0.119823,-0.446920,0.184366,0
2,-0.994489,-1.591641,-0.024982,-0.822024,0.251324,-0.513807,1.395402,-0.731683,0.358066,-2.170525,-1.670088,-1.703496,-1.589741,-1.820448,-0.128982,-2.149246,-0.040101,-0.270737,1.349780,0
3,-0.948301,-1.945523,0.176854,-1.527144,1.161379,-1.021953,3.419869,-0.981525,0.372224,-4.379766,-2.821361,-3.064039,-2.838498,-3.433336,0.124317,-3.691162,0.451554,-0.230795,2.148830,0
4,-1.387144,-2.214620,0.250335,-1.765739,1.753052,-1.131981,3.997494,-0.925982,0.295523,-4.918318,-2.741932,-3.532527,-3.296274,-3.970910,0.073046,-3.707865,0.606385,-0.075590,2.061627,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1389307,-1.984983,-1.983894,-1.988924,-1.983808,-1.983111,-1.984951,-1.982028,-1.977530,-1.981620,-1.966627,-1.989708,-1.986431,-1.991774,-1.986773,-1.985548,-1.977237,-1.986111,-1.987509,-1.983478,0
1389308,-1.971341,-1.971517,-1.985937,-1.983899,-1.984631,-1.986425,-1.986687,-1.990856,-1.988252,-1.988674,-1.985615,-1.982506,-1.991580,-1.991774,-1.992129,-1.989148,-1.984513,-1.987727,-1.988348,0
1389309,-1.975694,-1.976892,-1.981579,-1.985280,-1.986462,-1.985842,-1.988913,-1.996164,-1.990367,-2.005501,-1.980586,-1.981748,-1.982541,-1.987624,-1.989076,-1.995309,-1.983411,-1.984182,-1.988591,0
1389310,-1.992128,-1.992757,-1.981616,-1.986117,-1.986184,-1.983991,-1.985791,-1.986524,-1.985215,-1.994947,-1.981269,-1.985127,-1.976670,-1.979887,-1.980509,-1.987642,-1.984254,-1.981588,-1.983929,0


In [5]:
def redefine_window_target(df,threshold=2):
    if len(np.unique(df.iloc[:,-1])) == 1:
        target = df.iloc[0,-1]
    elif np.unique(df.iloc[:,-1],return_counts=True)[1][1] >= threshold*sample_rate:
        target = 1
    else:
        target = 0
    t_df = df.drop(columns = "is_seizure_target")
    window = pd.DataFrame(np.array(t_df))
    window["Target"] = target
    return window

In [6]:
#returning X_train, X_test, y_train, y_test
def prepare_x_y(df,threshold=2):
    #prepares the windows and x and y
    data=np.array([redefine_window_target(df.iloc[i:i+len_window ]) for i in range(0,len(df)-len_window, overlap*sample_rate)])
    X = data[:,:,:-1]
    y = data[:,:,-1]
    y=y.sum(axis=1)>=threshold*sample_rate
    #train and split indices 
    train_ind, test_ind = train_test_split(np.arange(767), test_size=0.2)

    X_train = X[train_ind,:,:]
    X_test = X[test_ind,:,:]
    y_train = y[train_ind]
    y_test = y[test_ind]
    return X_train, X_test, y_train,y_test

In [12]:
X_train, X_test, y_train,y_test=prepare_x_y(patient19)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.SimpleRNN(units=10, activation='tanh',input_shape=(2560, 19)))
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

# The compilation
model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy', 
        metrics=[ts.keras.metrics.Recall(),ts.keras.metrics.Precision(),"accuracy"])

# The fit
model.fit(X_train, y_train,
         batch_size=64,
         epochs=50, verbose=1)

results_DL =model.evaluate(X_test, y_test)
loss=results_DL[0]
recall = results_DL[1]
F1_score=2*((results_DL[2]* results_DL[1])/(results_DL[2]+ results_DL[1]))
accuracy = results_DL[3]


Epoch 1/50
10/10 [==============================] - 6s 419ms/step - loss: 0.7261 - recall_3: 0.2339 - precision_1: 0.3617 - accuracy: 0.5808
Epoch 2/50
10/10 [==============================] - 3s 338ms/step - loss: 0.7046 - recall_3: 0.2294 - precision_1: 0.3731 - accuracy: 0.5889
Epoch 3/50
10/10 [==============================] - 4s 375ms/step - loss: 0.6946 - recall_3: 0.2202 - precision_1: 0.3840 - accuracy: 0.5971
Epoch 4/50
10/10 [==============================] - 5s 465ms/step - loss: 0.6865 - recall_3: 0.1881 - precision_1: 0.3565 - accuracy: 0.5905
Epoch 5/50
10/10 [==============================] - 4s 401ms/step - loss: 0.6795 - recall_3: 0.1881 - precision_1: 0.3661 - accuracy: 0.5954
Epoch 6/50
10/10 [==============================] - 4s 353ms/step - loss: 0.6741 - recall_3: 0.1927 - precision_1: 0.3962 - accuracy: 0.6085
Epoch 7/50
10/10 [==============================] - 3s 347ms/step - loss: 0.6679 - recall_3: 0.1743 - precision_1: 0.4222 - accuracy: 0.6215
Epoch 8/50
10

In [17]:
print(F1_score)

0.14925373441048628
